In [46]:
from dash import Dash, html, dcc, Input, Output, callback, Patch, ALL, State, ctx, no_update
from dash.exceptions import PreventUpdate

import dash_cytoscape as cyto
from matplotlib import style
import json
import base64

app = Dash()

#Get the ID for the next node to be added to the list
#Goes through all ids and if they are integer finds the highest value and adds 1 to it, if none are integers returns 0. 
def getNextNodeID(elements):
    idList = []
    #print(elements)
    for element in elements:
        #print(element, ' element')
        if 'id' in element['data'].keys():
            if (element['data']['id'].isdigit()):
                idList.append(int(element['data']['id']))
    if len(idList) == 0:
        return 1
    else:
        return max(idList) + 1   
    
def addNewNode(elements):
    nodeDict = {}
    dataDict = {}
    classDict = {}
    nodeDict['id'] = str(getNextNodeID(elements))
    nodeDict['label'] = str(getNextNodeID(elements))
    nodeDict['function'] = "nseq"
    dataDict['data'] = nodeDict
    dataDict['classes'] = 'default ' + nodeDict['function']
    #styleDict = {}
    #styleDict['background-fit'] = 'cover'
    #styleDict['background-image'] = 'url(assets/nseq.jpg)'
    #dataDict['style'] = styleDict
    #print(elements, ' addNode ', elements.append(dataDict))
    
    elements.append(dataDict)
    #elements.append(classDict)
    print(elements)
    return elements

def addNewNodeNumber(elements, value):
    nodeDict = {}
    dataDict = {}
    nodeDict['id'] = str(value)
    nodeDict['label'] = str(value)
    nodeDict['function'] = "nseq"
    dataDict['data'] = nodeDict
    dataDict['classes'] = 'default ' + nodeDict['function']
    #styleDict = {}
    #styleDict['background-fit'] = 'cover'
    #styleDict['background-image'] = 'url(assets/nseq.jpg)'
    #dataDict['style'] = styleDict
    #print(elements, ' addNode ', elements.append(dataDict))
    elements.append(dataDict)
    print(elements)
    return elements


def removeNode(node, elements):
    #print('before remove ', elements)
    for element in elements:
      if 'id' in element['data'].keys():
        if ((node['id'] == "-7") or (node['id'] == "-5")):
            return elements
        if node['id'] == element['data']['id']:
            elements.remove(element)
    #print('after remove ', elements)
    return elements

def removeEdge(edge, elements):
    for element in elements:
        if 'source' in element['data'].keys():
            if (element['data']['source'] == edge['source']) and (element['data']['target'] == edge['target']):
                elements.remove(element)
    return elements

def addEdgeClasses(elements):
    for element in elements:
        if 'source' in element['data'].keys():
            element['classes'] = 'edge'
        if 'function' in element['data'].keys():
            element['classes'] = 'edge ' + element['data']['function']
    return elements



def isEdgeInElements(edge, elements):
    for element in elements:
        if 'source' in element['data'].keys():
            if (element['data']['source'] == edge['source']) and (element['data']['target'] == edge['target']):
                return True
    return False

def addNewEdge(node1, node2, elements):
    edgeDict = {}
    dataDict = {}
    edgeDict['source'] = node1['id']
    edgeDict['target'] = node2['id']
    dataDict['data'] = edgeDict
    dataDict['classes'] = 'edge'
    #styleDict = {}
    #styleDict['background-fit'] = 'cover'
    #styleDict['background-image'] = 'url(assets/nseq.jpg)'
    #dataDict['style'] = styleDict
    if isEdgeInElements(edgeDict, elements):
        return elements
    else:
        elements.append(dataDict)
        return elements

def setNodeColorToselected(node, elements):
    for element in elements:
        if 'id' in element['data'].keys():
            if element['data']['id'] == node['id']:
                element['classes'] = 'selected ' + element['data']['function']
    print(elements)
    return elements

def setNodeFunctionToOR(node, elements):
    for element in elements:
        if 'id' in element['data'].keys():
            if element['data']['id'] == node['id']:
                element['data']['function'] = 'nor'
                element['classes'] = 'default ' + element['data']['function']

    print(elements)
    return elements

def setNodeFunctionToXOR(node, elements):
    for element in elements:
        if 'id' in element['data'].keys():
            if element['data']['id'] == node['id']:
                element['data']['function'] = 'nxor'
                element['classes'] = 'default ' + element['data']['function']
    print(elements)
    return elements


def setNodeFunctionToAND(node, elements):
    for element in elements:
        if 'id' in element['data'].keys():
            if element['data']['id'] == node['id']:
                element['data']['function'] = 'nand'
                element['classes'] = 'default ' + element['data']['function']
    return elements


def setNodeFunctionToSEQ(node, elements):
    for element in elements:
        if 'id' in element['data'].keys():
            if element['data']['id'] == node['id']:
                element['data']['function'] = 'nseq'
                element['classes'] = 'default ' + element['data']['function']
    print(elements)
    return elements

def setIDColorToGreen(nodeID, elements):
    for element in elements:
        if 'id' in element['data'].keys():
            if element['data']['id'] == nodeID:
                element['classes'] = 'green ' + element['data']['function']
                #print("color set to green")
    return elements

def setIDColorToYellow(nodeID, elements):
    for element in elements:
        if 'id' in element['data'].keys():
            if element['data']['id'] == nodeID:
                element['classes'] = 'yellow ' + element['data']['function']
                #print("color set to yellow")
    return elements

def unselectNode(node, elements):
    for element in elements:
      if 'id' in element['data'].keys():
        if element['data']['id'] == node['id']:
            if 'classes' in element.keys():
                element['classes'] = "default " + element['data']['function']
    print(elements)
    return elements

def unselectNodeByID(id, elements):
    for element in elements:
      if 'id' in element['data'].keys():
        if element['data']['id'] == id:
            if 'classes' in element.keys():
                element['classes'] = "default " + element['data']['function']
    print(elements)
    return elements


def processTupleList(tupleList, elements):
    tlID = []
    if (tupleList==None):
        return elements
    else:
        tl = tupleList.split('t(')
        #print(tl)
        del tl[0]
        #print(tl)
        for tupleT in tl:
            tuples = tupleT.split(',')
            #print(tuples)
            nodeID = tuples[0]
            tlID.append(nodeID)
            indicator = tuples[1].split(")")[0]
            #print(nodeID, "   s   ", indicator)
            if (int(indicator) == 1):
                #print("node id is one")
                elements = setIDColorToGreen(str(int(nodeID)), elements)
            elif (int(indicator) == -1):
                #print("node id is minus one")
                elements = setIDColorToYellow(str(int(nodeID)), elements)
            #print("node indicator is ", int(indicator), ' node id is ', int(nodeID))
    for element in elements:
        if 'function' in element['data'].keys():
            if not (element['data']['id'] in tlID):
                elements = unselectNodeByID(element['data']['id'], elements)
    return elements

#def getIdByLabel(label, elements):
#    for element in elements:
#        if 'function' in element['data'].keys():
#            if element['data']['label'] == label:
#                return 
    
#def getLabelById(nodeid, elements):
    

#encode labels to id's
#def encodeSequence(sequence, elements):
#    for label in sequence:
#only encode S and F
def encodeSequence(sequence):
    outputSequence = []
    for label in sequence: 
        if ((label == 'S') or (label == 's')): 
            outputSequence.append('-5')
        elif ((label == 'F') or (label == 's')):
            outputSequence.append('-7')
        else:
            outputSequence.append(label)
    return outputSequence

def decodeSequence(sequence):
    outputSequence = []
    for label in sequence: 
        if (label == '-5'): 
            outputSequence.append('S')
        elif (label == '-7'):
            outputSequence.append('F')
        else:
            outputSequence.append(label)
    return outputSequence
        
def encodeInputAction(item):
    if ((item == 'S') or (item == 's')):
        return '-5'
    elif ((item == 'F') or (item == 'f')):
        return '-7'
    else:
        return item

def decodeInputAction(item):
    if (item == '-5'):
        return 'S'
    elif (item == '-7'):
        return 'F'
    else:
        return item
    

    

inlineButtonStyle = {'background-color':'lightgray', 'margin':'5px', 'display': 'inline-block','width':'45%', 'padding-top':'5px', 'padding-bottom':'5px'}
buttonStyle = {'background-color':'lightgray', 'margin':'5px', 'align-items':'center', 'width':'45%', 'padding-top':'5px', 'padding-bottom':'5px'}
openButtonStyle = {'background-color':'lightgray', 'margin':'5px', 'align-items':'center', 'width':'45%'}
inputStyle = {'margin':'5px', 'width':'43.5%', 'padding-top':'5px', 'padding-bottom':'5px'}
invisibleButtonStyle = {'display': 'none'}
activeButtonStyle = {'background-color':'coral', 'margin':'5px', 'padding-top':'5px', 'padding-bottom':'5px','width':'45%'}
itemBox = {'border-style':'solid','border-color':'gray', 'margin':'5px', 'text-align':'center', 'padding-bottom':'10px', 'padding-top':'10px', 'background-color':'Aquamarine'}
textStyle = {'margin':'5px', 'padding':'0px', 'display': 'inline-block'}

app.layout = [
    dcc.Store(id='nodeSelectedPreviously'),
    dcc.Store(id='nodeSelectedForDeletion'),
    dcc.Store(id='selectedEdge'),
    dcc.Store(id='addEdgeFLag'),
    dcc.Store(id="functionFlag"),
    dcc.Store(id="sequenceFlag"),
    dcc.Store(id="realTimeFlag"),
    dcc.Store(id="config"),
    dcc.Store(id='removeNodeFlag'),
    dcc.Store(id='edgeFlags'),
    html.Div(children = [
        html.Div(
            cyto.Cytoscape(
                id='cytoscape',
                elements=addEdgeClasses([{"data": {"id": "5", "label": "5", "function": "nseq", "url": "nseq.jpg"}, "position": {"x": -123.98678613466984, "y": -25.75959836336593}}, {"data": {"id": "1", "label": "1", "function": "nseq"}, "position": {"x": -199.69687989601323, "y": -27.542949942975344}}, {"data": {"id": "2", "label": "2", "function": "nor"}, "position": {"x": 22.671122520389762, "y": -23.69302613659202}}, {"data": {"id": "3", "label": "3", "function": "nxor"}, "position": {"x": 122.9677742934759, "y": -20.47176758837865}}, {"data": {"id": "-5", "label": "S", "function": "nseq"}, "position": {"x": -268.32100254356936, "y": -27.322884268868464}}, {"data": {"id": "8", "label": "8", "function": "nseq"}, "position": {"x": -50.57717883108286, "y": -25.519417278496693}}, {"data": {"id": "6", "label": "6", "function": "nor"}, "position": {"x": 71.0986485417034, "y": -73.12253935726854}}, {"data": {"id": "4", "label": "4", "function": "nseq"}, "position": {"x": 72.49248658272701, "y": 30.514112247201776}}, {"data": {"id": "9", "label": "9", "function": "nseq"}, "position": {"x": 122.31685604609123, "y": -73.56298803233294}}, {"data": {"id": "11", "label": "11", "function": "nseq"}, "position": {"x": 195.57103468290944, "y": -16.46038274565095}}, {"data": {"id": "-7", "label": "F", "function": "nseq"}, "position": {"x": 273.98413567618877, "y": -17.929007542077223}}, {"data": {"source": "-5", "target": "1", "id": "bb369971-020e-4fdb-ba5d-0b0ec9d0aeb7"}}, {"data": {"source": "1", "target": "5", "id": "266c201c-5b94-4a61-8867-b97e24e48daa"}}, {"data": {"source": "5", "target": "8", "id": "e5309d24-af88-44fc-9338-b5e9b10f6b2c"}}, {"data": {"source": "8", "target": "2", "id": "904fb9ce-a149-45ea-be8e-7022423d5212"}}, {"data": {"source": "2", "target": "4", "id": "c989d3c7-2621-452d-8da7-ce7e938d5af6"}}, {"data": {"source": "4", "target": "3", "id": "e31c13f4-71c2-4dab-808a-b6eeac11ce8a"}}, {"data": {"source": "3", "target": "6", "id": "cbdd6c02-619e-4a07-ade3-f17550b88c43"}}, {"data": {"source": "6", "target": "2", "id": "eda4823d-b238-465f-925d-0a12b583ece2"}}, {"data": {"source": "3", "target": "9", "id": "2ed0dca2-39a5-441e-b85f-08644beda884"}}, {"data": {"source": "9", "target": "6", "id": "c553eded-d4e8-4988-a5de-31bbe3f1d671"}}, {"data": {"source": "3", "target": "11", "id": "c7b5abf8-ba12-4969-8c37-d7ea8efd463e"}}, {"data": {"source": "11", "target": "-7", "id": "b0d49908-88ba-4b5b-848e-1d20686fa233"}}
                        ]),
                layout={'name': 'preset'},
                style={'width': '97%','height': '550px', 'background-color': 'honeydew', 'border-style': 'groove', 'margin': '10px 0px 0px 0px'},
                stylesheet=[
                    {
                        'selector': 'default',
                        'style': {
                            #'background-color': '#969696',
                            'content': 'data(label)',
                            #'background-fit': 'cover',
                            #'background-image': 'url(assets/data(function).jpg)',
                        }
                    },
                    {
                        'selector': '.edge',
                        'style': {
                            'target-arrow-shape': 'triangle',
                            'arrow-scale': "2",
                            'curve-style': 'bezier',
                        }
                    },
                    {
                        'selector': '.default',
                        'style': {
                            #'background-color': '#969696',
                            #'background-opacity': 0.5,
                            'content': 'data(label)',
                            #'content': 'dat#a(function)',
                            #'background-fit': 'cover',
                            #'background-image': 'url(assets/nseq.jpg)'
                            #'background-image-opacity': 0.5,
                        }
                    },
                    {
                        'selector': '.selected',
                        'style': {
                            #'background-color': '#0099ff',
                            #'opacity': 0.5,
                            'content': 'data(label)',
                            'overlay-color' : '#0099ff',
                            'overlay-opacity': '0.5',
                            'overlay-shape': 'ellipse',
                            'overlay-padding': '1px',
                            #'border-width': 4,
                            #'border-color': 'green',
                            #'background-fit': 'cover',
                            #'background-image': 'url(assets/nseq.jpg)',
                            #'background-opacity': 0.5,
                            #'background-image-opacity': 0.5,
                        }
                    },
                    {
                        'selector': '.green',
                        'style': {
                            #'background-color': '#008000',
                            'content': 'data(label)',
                            'overlay-color' : '#008000',
                            'overlay-opacity': '0.5',
                            'overlay-shape': 'ellipse',
                            'overlay-padding': '1px',
                        }
                    },
                    {
                        'selector': '.yellow',
                        'style': {
                            #'background-color': '#FFFF00',
                            'content': 'data(label)',
                            'overlay-color' : '#FFFF00',
                            'overlay-opacity': '0.5',
                            'overlay-shape': 'ellipse',
                            'overlay-padding': '1px',
                        }
                    },
                    {
                        'selector': '.nor',
                        'style': {
                            'background-fit': 'cover',
                            'background-image': 'url(assets/or.png)',
                        }
                    },
                    {
                        'selector': '.nand',
                        'style': {
                            'background-fit': 'cover',
                            'background-image': 'url(assets/and.png)',
                        }
                    },
                    {
                        'selector': '.nxor',
                        'style': {
                            'background-fit': 'cover',
                            'background-image': 'url(assets/xor.png)',
                        }
                    },
                    {
                        'selector': '.nseq',
                        'style': {
                            'background-fit': 'cover',
                            'background-image': 'url(assets/seq.png)',
                        }
                    }
                ]
            ),
            style={'width': '70%', 'height': 'inherit', 'display': 'inline-block', 'vertical-aligh': 'top'},
        ),
            html.Div(
            dcc.Tabs(children=[
                dcc.Tab(label="Edit Pattern", children=[
                    html.Br(),
                    html.Button(style = buttonStyle, id='addNode_',children='Add a new Graph Node'),
                    dcc.Input(style=inputStyle, id="newNodeNumber", type="number", value=None, placeholder="New Node ID"),
                    html.Br(),
                    html.Br(),
                    html.Button(style = buttonStyle, id='RemoveNode', children='Remove a Specific Graph Node'),
                    html.Button(style = buttonStyle, id='RemoveEdge', children='Remove a Specific Graph Edge'),
                    html.Br(),
                    html.Br(),
                    html.Button(style = buttonStyle, id="addEdges", children="Start Adding Edges"),
                    html.Br(),
                    html.Br(),
                    html.Button(style = buttonStyle, id="functionToggle", children="Set Node Function"),
                    html.Br(),
                    html.Button( id="funcOR", children = "Set node to OR" , style = dict(display='none')),
                    html.Button( id="funcXOR", children = "Set node to XOR" , style = dict(display='none')),
                    html.Button( id="funcAND", children = "Set node to AND" , style = dict(display='none')),
                    html.Button( id="funcSEQ", children = "Set node to SEQUENTIAL" , style = dict(display='none')),
                    #html.Br(),
                    html.Br(),
                    html.Button(style = buttonStyle, id="resetGraph", children= "Reset Graph"),
                    html.Br(),
                    html.Br(),
                    html.Div(children=[
                        dcc.Upload(id="upload", 
                                    children=html.Button(style = {'width':'100%','background-color':'lightgray', 'margin':'5px', 'display': 'inline-block', 'width':'100%', 'padding-top':'5px', 'padding-bottom':'5px'}, 
                                                         children = "Upload Graph", 
                                                         id = "uploadBTN"), 
                                                         ),
                        ], style={'display': 'inline-block', 'width':'45%', 'margin-right':'5px'},
                    ),
                    html.Div(children=[
                    html.Button(style = {'background-color':'lightgray', 'margin':'5px','width':'100%', 'display': 'inline-block', 'padding-top':'5px', 'padding-bottom':'5px'}, children = "Download Graph", id="downloadBTN"),
                    dcc.Download(id="download-graph"),
                    ], style={'display': 'inline-block','width':'45%', 'margin-left':'5px'}),
                    ], style={'padding': '10px 0px 10px', 'vertical-align':'middle'}, selected_style={'padding': '10px 0px 10px', 'vertical-align':'middle','background-color' : 'darkgrey'},),
                dcc.Tab(style={'padding': '10px 0px 10px', 'vertical-align':'middle'}, selected_style={'padding': '10px 0px 10px', 'vertical-align':'middle','background-color' : 'darkgrey'}, label="Edit Sequence", children=[
                    html.Br(),
                    dcc.Input(id="sequenceInput", placeholder="Input sequence", style=inputStyle),
                    html.Br(),
                    html.Br(),
                    html.Button(style = buttonStyle, id="sequenceToggle", children="Start Adding the Sequence"),
                    html.Br(),
                    html.Br(),
                    html.Div(children=[
                        dcc.Upload(
                            id="uploadSequence", 
                            children = [
                                html.Button(
                                    style = {'width':'100%','background-color':'lightgray', 'margin':'5px', 'display': 'inline-block', 'width':'100%', 'padding-top':'5px', 'padding-bottom':'5px'}, 
                                    id="uploadSequenceBTN", 
                                    children = "Upload Sequence")
                                ]
                            ),
                        ], style={'display': 'inline-block', 'width':'45%', 'margin-right':'5px'}),

                    html.Button(
                        style = buttonStyle, 
                        id="downloadSequenceBTN", 
                        children="Download Sequence"
                        ),

                    
                    dcc.Download(id="download-sequence"),
                    ]),
                dcc.Tab(style={'padding': '10px 0px 10px', 'vertical-align':'middle'}, selected_style={'padding': '10px 0px 10px', 'vertical-align':'middle','background-color' : 'darkgrey'},label="Supervision", children=[
                    #html.Br(),
                    html.Div(children=[
                    html.H4(children="Conformance Checking", style=textStyle),
                    html.Br(),
                    html.Button(style = buttonStyle, id="checkCorrectness", children="Check Conformance"),
                    ], style=itemBox),
                    #html.Br(),
                    #html.Br(),
                    html.Div(children=[
                    html.H4(children="Step By Step Supervision", style=textStyle),
                    html.Br(),
                    html.Button(style = inlineButtonStyle, id="allSteps", children="Make all Steps"),
                    html.Button(style = inlineButtonStyle, id="makeStep", children="Make one Step"),
                    html.Div(id="specialDivSequence", style={'display':'none'}),
                    ],style=itemBox),
                    #html.Br(),
                    #html.Br(),
                    html.Div(children=[
                    html.H4(children="Interactive Supervision", style=textStyle),
                    html.Br(),
                    html.Button(style = buttonStyle, id="realTime", children="Start Interactive Supervision"),
                    html.Br(),
                    dcc.Input(style=inputStyle,id='manualInput', placeholder='Manual Action Input'),
                    html.Button(style = buttonStyle, id='manualInputBTN', children = 'Input Manual Action'),
                    ], style=itemBox),
                    html.Div(children=[
                        html.H4(children="Input Sequence:", style=textStyle),
                        html.Br(),
                        html.Div(id="sequence", style = dict(display="none")),
                        html.Div(id="divSequence", style=textStyle),
                        ], style=itemBox),
                    html.Div(children=[
                            html.H4(id="correctActionsLabel", children="Correct Actions:  ", style=textStyle),
                            html.Br(),
                            html.H4(id="correctActions", style=textStyle),
                        ], 
                        style=itemBox),
                    html.Div(children=[
                        html.H4(id="incorrectActionsLabel", children="Incorrect Actions: ", style=textStyle),
                        html.Br(),
                        html.H4(id="incorrectActions", style=textStyle),
                        ], style=itemBox),
                    #html.Br(),
                    html.Div(children=[
                        html.H5(id="indicatorsLabel", children="indicators", style={'display': 'none'}),
                        html.H5(id="indicators", style={'display': 'none'}),
                        html.H4(id="statusLabel", children="Status: ", style=textStyle),
                        html.Br(),
                        html.H4(id="correctness", style=textStyle),
                        ], style = itemBox),
                    ]),
                # dcc.Tab(label="Save/Upload Graph", children=[   
                #     html.Button(style = buttonStyle, "Download Graph", id="downloadBTN"),
                #     dcc.Download(id="download-graph"),
                #     dcc.Upload(id="upload", children=html.Button(style = buttonStyle, "Upload Graph", id = "uploadBTN")),
                #     ]),
                dcc.Tab(style={'padding': '10px 0px 10px', 'vertical-align':'middle'}, selected_style={'padding': '10px 0px 10px', 'vertical-align':'middle', 'background-color' : 'darkgrey'},label="Examples", children=[
                    
                    ])
                ], style={'width': '100%'}),
            style={'margin':'10px 0px 0px 0px', 'width': '29%', 'height': 'inherit', 'display': 'inline-block', 'vertical-align': 'top', 'background-color': 'honeydew', 'border-style':'solid','border-color':'gray',},
        ),],
        style={'margin': '20px 0px 0px 0px', 'height': '100%' },
    )
]



#manual real time manual action Input button
@callback(
        Output('manualInput', 'value', allow_duplicate=True),
        Output('cytoscape', 'elements', allow_duplicate=True),
        Output('correctActions', 'children', allow_duplicate=True),
        Output('incorrectActions', 'children', allow_duplicate=True),
        Output('correctness', 'children', allow_duplicate=True),
        Output('config','data', allow_duplicate=True),
        Input('manualInputBTN', 'n_clicks'),
        State('manualInput', 'value'),
        State('realTimeFlag', 'data'),
        State('config', 'data'),
        State('cytoscape','elements'),
        prevent_initial_call=True,
        
)
def manualInputBTN(clicks,inputValue,realTime,config,elements):
    if (realTime != None) and (realTime == True):
        previousCorrect = ""
        previousIncorrect = ""
        if (config != None):
            config = makeStep( "a(" + inputValue + ") | " + getTupleList(config) + " | " + getCorrectList(config) + " | " + getIncorrectList(config) + " | " + elementsToGraph(elements) + " | " + getStatus(config) + " " )
            
            if ("-7" in getCorrectList(config)):
                config = makeStep(config)
        else:
            config = makeStep(generateTerm(elements,[inputValue]))
        return None, processTupleList(getTupleList(config), elements), getCorrectList(config), getIncorrectList(config), getStatus(config), config
    else:
        #Dash.exceptions.PreventUpdate
        return no_update, no_update, no_update, no_update, no_update, no_update
        

#make all steps
@callback(
        Output("divSequence", "children", allow_duplicate=True),
        Output("correctActions", "children", allow_duplicate=True),
        Output("incorrectActions", "children", allow_duplicate=True),
        Output("correctness", "children", allow_duplicate=True),
        Output("cytoscape", "elements", allow_duplicate=True),
        Input("allSteps", "n_clicks"),
        State("sequenceInput", "value"),
        State("divSequence", "children"),
        State("cytoscape", "elements"),
        prevent_initial_call=True,
)
def allSteps(click, sequence, divSequence, elements):
    config = generateTerm(elements, encodeSequence(sequence.split(",")))
    status = getStatus(config)
    correct = getCorrectList(config)
    incorrect = getIncorrectList(config)
    tuple = getTupleList(config)
    while (config != None):
        oldConfig = config
        status = getStatus(config)
        correct = getCorrectList(config)
        incorrect = getIncorrectList(config)   
        tuple = getTupleList(config)
        config = makeStep(oldConfig)
        if (config != None):
            if (gentNewCorrectNumber(oldConfig, config)==None):
                divSequence = makeNewNumberRed(divSequence)
            if (gentNewIncorrectNumber(oldConfig, config)==None):
                divSequence = makeNewNumberGreen(divSequence)
    return divSequence, correct, incorrect, status, processTupleList(tuple, elements)

        

#reset graph
@callback(
        Output("cytoscape", "elements", allow_duplicate=True),
        Input("resetGraph", "n_clicks"),
        prevent_initial_call=True,
)
def resetGraph(clikcs):
    return []

#upload sequence button callback
@callback(
        Output("sequenceInput", "value", allow_duplicate=True),
        Input("uploadSequence", "contents"),
        prevent_initial_call=True,
)
def uploadSequence(content):
    content = base64.b64decode(content.split(',')[1])
    content = json.loads(content)
    content = ",".join(content)
    return content

#download sequence button callback
@callback(
        Output("download-sequence", "data", allow_duplicate=True),
        Input("downloadSequenceBTN", "n_clicks"),
        State("sequenceInput", "value"),
        prevent_initial_call=True,
        )
def downloadSequence(clicks, sequence):
    slist = sequence.split(",")
    jsonElements = json.dumps(slist)
    return {'content': jsonElements, 'filename': 'sequence.json'}

#"Add Node" button callback    
@callback(
    Output("cytoscape", "elements",  allow_duplicate=True),
    Output("newNodeNumber", "value", allow_duplicate=True),
    State("cytoscape", "elements"),
    Input("addNode_", "n_clicks"),
    State("newNodeNumber", "value"),
    prevent_initial_call=True,
)
def addNewNodeCallback(cyto, n_cl, value):
    if value == None:
        return addNewNode(cyto), None
    else:
        return addNewNodeNumber(cyto, value), None

#Callback for adding edges by tapping two nodes
@callback(
    Output("removeNodeFlag", "data", allow_duplicate=True),
    Output("RemoveNode", "children", allow_duplicate=True),
    Output("cytoscape", "elements", allow_duplicate=True),
    Output("nodeSelectedPreviously", "data", allow_duplicate=True),
    Output('nodeSelectedForDeletion', "data", allow_duplicate=True),
    Output("sequenceInput", "value", allow_duplicate=True),
    Output("sequence", "children", allow_duplicate=True),
    Output("funcOR", "style", allow_duplicate=True),
    Output("funcXOR", "style", allow_duplicate=True),
    Output("funcAND", "style", allow_duplicate=True),
    Output("funcSEQ", "style", allow_duplicate=True),
    Output("correctActions", "children", allow_duplicate=True),
    Output("incorrectActions", "children", allow_duplicate=True),
    Output("correctness", "children", allow_duplicate=True),
    Output("config", "data", allow_duplicate=True),
    Output("specialDivSequence", "children", allow_duplicate=True),
    State("nodeSelectedPreviously", "data"),
    State("cytoscape", "elements"),
    State("RemoveNode", "children"),
    Input("cytoscape", "tapNodeData"),
    State("removeNodeFlag", "data"),
    State("addEdgeFLag", "data"),
    State("nodeSelectedForDeletion", "data"),
    State("sequenceFlag", "data"),
    State("sequenceInput", "value"),
    State("functionFlag", "data"),
    State("realTimeFlag", "data"),
    State("correctActions", "children"),
    State("incorrectActions","children"),
    State("config", "data"),
    State("sequence", "children"),
    State("specialDivSequence", "children"),
    prevent_initial_call=True,
)
def tapNodeAddEdge(prevNode, elements, rmNodeButton, tappedNode, flags, addEdgeFLag, nodeSelectedForDeletion, seqFlag, sequence, functionFlag, realTime, correct, incorrect, config, shownsequence,specialDivSequence):
    if (realTime != None) and (realTime == True):
        previousCorrect = ""
        previousIncorrect = ""
        if (config != None):
            oldConfig = config
            config = makeStep( "a(" + tappedNode['id'] + ") | " + getTupleList(config) + " | " + getCorrectList(config) + " | " + getIncorrectList(config) + " | " + elementsToGraph(elements) + " | " + getStatus(config) + " " )
            if (sequence == None):
                sequence = str(decodeInputAction(tappedNode['id']))
            else:
                sequence = sequence + ',' + decodeInputAction(tappedNode['id'])
            if ("-7" in getCorrectList(config)):
                config = makeStep(config)
        else:
            oldConfig = generateTerm(elements,[tappedNode['id']])
            config = makeStep(generateTerm(elements,[tappedNode['id']]))
            if (sequence == None):
                sequence = str(decodeInputAction(tappedNode['id']))
        print('oldconfig ', oldConfig)
        print('config ', config)
        if (specialDivSequence != None):
            if (config != None):
                if (gentNewCorrectNumber(oldConfig, config)==None):
                    a = html.H4(children=str(decodeInputAction(tappedNode['id'])), style={'color': '#FF0000',  'margin-top':'5px', 'margin-bottom':'5px', 'margin-left':'0px', 'margin-right':'0px', 'padding':'0px', 'display': 'inline-block'}, title="unprocessed")
                    b = html.H4(children=",", style={'color': '#000000',  'margin-top':'5px', 'margin-bottom':'5px', 'margin-left':'0px', 'margin-right':'0px', 'padding':'0px', 'display': 'inline-block'}, title="comma")
                    specialDivSequence.append(b)
                    specialDivSequence.append(a)
                if (gentNewIncorrectNumber(oldConfig, config)==None):
                    a = html.H4(children=str(decodeInputAction(tappedNode['id'])), style={'color': '#008000',  'margin-top':'5px', 'margin-bottom':'5px', 'margin-left':'0px', 'margin-right':'0px', 'padding':'0px', 'display': 'inline-block'}, title="unprocessed")
                    b = html.H4(children=",", style={'color': '#000000',  'margin-top':'5px', 'margin-bottom':'5px', 'margin-left':'0px', 'margin-right':'0px', 'padding':'0px', 'display': 'inline-block'}, title="comma")
                    specialDivSequence.append(b)
                    specialDivSequence.append(a)
        else:
            if (config != None):
                if (gentNewCorrectNumber(oldConfig, config)==None):
                    a = html.H4(children=str(decodeInputAction(tappedNode['id'])), style={'color': '#FF0000',  'margin-top':'5px', 'margin-bottom':'5px', 'margin-left':'0px', 'margin-right':'0px', 'padding':'0px', 'display': 'inline-block'}, title="unprocessed")
                    b = html.H4(children=",", style={'color': '#000000',  'margin-top':'5px', 'margin-bottom':'5px', 'margin-left':'0px', 'margin-right':'0px', 'padding':'0px', 'display': 'inline-block'}, title="comma")
                    #specialDivSequence.append(b)
                    specialDivSequence = [a]
                if (gentNewIncorrectNumber(oldConfig, config)==None):
                    a = html.H4(children=str(decodeInputAction(tappedNode['id'])), style={'color': '#008000',  'margin-top':'5px', 'margin-bottom':'5px', 'margin-left':'0px', 'margin-right':'0px', 'padding':'0px', 'display': 'inline-block'}, title="unprocessed")
                    b = html.H4(children=",", style={'color': '#000000',  'margin-top':'5px', 'margin-bottom':'5px', 'margin-left':'0px', 'margin-right':'0px', 'padding':'0px', 'display': 'inline-block'}, title="comma")
                    #specialDivSequence.append(b)
                    specialDivSequence = [a]
        return flags, rmNodeButton,  processTupleList(getTupleList(config), elements), None, None, sequence, sequence, dict(display='none'), dict(display='none'), dict(display='none'), dict(display='none'), getCorrectList(config).replace('a(-5)', 'a(S)').replace('a(-7)','a(F)'), getIncorrectList(config).replace('a(-5)', 'a(S)').replace('a(-7)','a(F)'), getStatus(config), config, specialDivSequence
    else:
        if (functionFlag !=None) and (functionFlag == True):
            store = tappedNode
            funcButton = activeButtonStyle
            return flags, rmNodeButton, elements, store, nodeSelectedForDeletion, sequence, sequence, funcButton, funcButton, funcButton, funcButton, None, None, None, config,specialDivSequence
        else:
            if (seqFlag != None) and (seqFlag == True):
                if sequence == None:
                    sequence = tappedNode['id']
                else:
                    sequence = sequence + "," + tappedNode['id']
                return flags, rmNodeButton, elements, None, nodeSelectedForDeletion, sequence, sequence, dict(display='none'), dict(display='none'), dict(display='none'), dict(display='none'), None, None, None, config,specialDivSequence
            else:
                if (addEdgeFLag == None) or (addEdgeFLag == False):
                    store = None
                    if (flags != None) and ('removeNode' in flags.keys()) and (flags['removeNode'] == True):
                        elements = removeNode(tappedNode, elements)
                        flags['removeNode'] = False
                        rmNodeButton = "Remove a Specific Graph Node"
                        return flags, rmNodeButton, elements, store, nodeSelectedForDeletion, sequence, sequence, dict(display='none'), dict(display='none'), dict(display='none'), dict(display='none'), None, None, None, config,specialDivSequence
                    elif nodeSelectedForDeletion == None:
                        nodeSelectedForDeletion = tappedNode
                        elements = setNodeColorToselected(tappedNode, elements)
                        rmNodeButton = "Remove the Selected Graph Node"
                    else:
                        elements = unselectNode(tappedNode, elements)
                        elements = unselectNode(nodeSelectedForDeletion, elements)
                        nodeSelectedForDeletion = None
                        rmNodeButton = "Remove a Specific Graph Node"
                    return flags, rmNodeButton, elements, None, nodeSelectedForDeletion, sequence, sequence, dict(display='none'), dict(display='none'), dict(display='none'), dict(display='none'), None, None, None, config,specialDivSequence
                else:
                    #print(elements)
                    store = None
                    #print(flags)
                    if (flags != None) and ('removeNode' in flags.keys()) and (flags['removeNode'] == True):
                            elements = removeNode(tappedNode, elements)
                            flags['removeNode'] = False
                            rmNodeButton = "Remove a Specific Graph Node"
                            return flags, rmNodeButton, elements, store, sequence, dict(display='none'), dict(display='none'), dict(display='none'), dict(display='none'), None, None, None, config,specialDivSequence
                    else:
                        if prevNode == None:
                            store = tappedNode
                            elements = setNodeColorToselected(tappedNode, elements)
                            rmNodeButton = "Remove the Selected Graph Node"
                        else:
                            elements = unselectNode(prevNode, elements)
                            elements = addNewEdge(prevNode, tappedNode, elements)
                            elements = unselectNode(tappedNode, elements)
                            store = None   
                            rmNodeButton = "Remove a Specific Graph Node"
                    return flags, rmNodeButton, elements, store, nodeSelectedForDeletion, sequence, sequence, dict(display='none'), dict(display='none'), dict(display='none'), dict(display='none'), None, None, None, config,specialDivSequence

#specialDivSequence callback
@callback(
        Output("divSequence", "children", allow_duplicate=True),
        Input("specialDivSequence", "children"),
        prevent_initial_call=True,
)
def changeDivSequence(specialDivSequence):
    return specialDivSequence

#"Remove Node" buttton callback
@callback(
    Output("cytoscape", "elements", allow_duplicate=True),
    Output("removeNodeFlag", "data", allow_duplicate=True),
    Output("RemoveNode", "children", allow_duplicate=True),
    Output("nodeSelectedPreviously", "data", allow_duplicate=True),
    Output('nodeSelectedForDeletion', "data", allow_duplicate=True),
    State("cytoscape", "elements"),
    State("removeNodeFlag", "data"),
    State("nodeSelectedPreviously", "data"),
    Input("RemoveNode", "n_clicks"),
    State("RemoveNode", "children"),
    State("nodeSelectedForDeletion", "data"),
    prevent_initial_call=True,
    )
def removeNodePressed(elements, flags, selectedNode, clicks, rmNodeButton, nodeSelectedForDeletion):
    if nodeSelectedForDeletion != None:
        elements = removeNode(nodeSelectedForDeletion, elements)
        rmNodeButton = "Remove a Specific Graph Node"
        nodeSelectedForDeletion = None
    elif selectedNode != None:
        elements = removeNode(selectedNode, elements)
        rmNodeButton = "Remove a Specific Graph Node"
        selectedNode = None
    else:
        dictF = {}
        dictF['removeNode'] = True
        flags = dictF
        rmNodeButton = "Select Graph Node To Remove"
    return elements, flags, rmNodeButton, selectedNode, nodeSelectedForDeletion

#"Remove Edge" button callback
@callback(
    Output("RemoveEdge", "children", allow_duplicate=True),
    Output("cytoscape", "elements", allow_duplicate=True),
    Output("edgeFlags", "data", allow_duplicate=True),
    Output("selectedEdge", "data", allow_duplicate=True),
    Input("RemoveEdge", "n_clicks"),
    State("edgeFlags", "data"),
    State("cytoscape", "elements"),
    State("selectedEdge", "data"),
    prevent_initial_call=True,
)
def removeEdgeButton(button, edgeFlags, elements, selectedEdge):
    if (edgeFlags == None) or (edgeFlags == False):
        if (selectedEdge == None):
            flag = True
            edgeFlags = flag
            buttonMSG = "Select Edge to be Removed"
        else:
            elements = removeEdge(selectedEdge, elements)
            selectedEdge = None
            buttonMSG = "Remove a Specific Graph Edge"
    else:
        edgeFlags = False
        buttonMSG = "Remove a Specific Graph Edge"
    return buttonMSG, elements, edgeFlags, selectedEdge

#Callback on tapped/clicked edge
@callback(
    Output("RemoveEdge", "children", allow_duplicate=True),
    Output("cytoscape", "elements", allow_duplicate=True),
    Output("edgeFlags", "data", allow_duplicate=True),
    Output("selectedEdge", "data", allow_duplicate=True),
    State("edgeFlags", "data"),
    State("cytoscape", "elements"),
    Input("cytoscape", "tapEdgeData"),
    prevent_initial_call=True,
)
def selctedEdge(flags, elements, edge):
    selectedEdge = None
    if (flags != None) and (flags == True):
        elements = removeEdge(edge, elements)
        selectedEdge = None
        buttonMSG = "Remove a Specific Graph Edge"
        flags = False
    elif (flags == None) or (flags == False):
        selectedEdge = edge
        buttonMSG = "Remove the Selected Edge"
    return buttonMSG, elements, flags, selectedEdge


#Button to start/stop adding edges
@callback(
    Output("addEdgeFLag", "data"),
    Output("addEdges", "children"),
    Input("addEdges", "n_clicks"),
    State("addEdgeFLag", "data"),
    prevent_initial_call=True,
)
def edgeAddButtonPressed(clicks, edgesToggle):
    if (edgesToggle==None) or (edgesToggle==False):
        edgesToggle = True
        buttonMSG = "Stop Adding Edges"
    elif(edgesToggle == True):
        edgesToggle = False
        buttonMSG = "Start Adding Edges"
    return edgesToggle, buttonMSG

#callback to download the list of elements as a json
@callback(
    Output("download-graph", "data"),
    Input("downloadBTN", "n_clicks"),
    State("cytoscape", "elements"),
    State("sequenceInput","value"),
    prevent_initial_call=True,
)
def downloadGraphButtonPushed(clicks, elements, sequence):
    sdict = {}
    sdict['sequence'] = sequence
    seqDict = {}
    seqDict['data']= sdict
    elements.append(seqDict)
    jsonElements = json.dumps(elements)
    #print(jsonElements)
    return {'content': jsonElements, 'filename': 'graph.json'}

#callback to upload a list of elements as a json
@callback(
    Output("cytoscape", "elements", allow_duplicate=True),
    Output("sequenceInput", "value", allow_duplicate=True),
    Input("upload", "contents"),
    prevent_initial_call=True,
)
def uploadGraph(content):
    #print(content)
    content = base64.b64decode(content.split(',')[1])
    sequence = json.loads(content)[-1]['data']['sequence']
    return (addEdgeClasses((json.loads(content)[:-1]))), sequence
    
#check correctness button
@callback(
    Output("correctness", "children", allow_duplicate=True),
    State("cytoscape", "elements"),
    Input("checkCorrectness", "n_clicks"),
    State("sequenceInput","value"),
    prevent_initial_call=True,
)
def checkCorrectnessButton(elements,clicks,sequence):
    #tempList = [1,2,3,4,5,6]
    tempList = encodeSequence(sequence.split(","))
    #print(tempList)
    correct = checkCorrectness(generateTerm(elements,tempList))
    if correct:
        return "Correct sequence"
    else:
        return "Incorrect sequence"

#add sequence button
@callback(
    Output("sequenceFlag", "data", allow_duplicate=True),
    Output("sequenceToggle", "children", allow_duplicate=True),
    Input("sequenceToggle", "n_clicks"),
    State("sequenceFlag", "data"),
    State("sequenceToggle", "children"),
    prevent_initial_call=True,
)
def addSequenceButton(clicks, flag, button):
    if ((flag == None) or (flag == False)):
        flag = True
        button = "Finish Adding the Sequence"
    else:
        flag = False
        button = "Start Adding the Sequence"
    return flag, button

#make step button
@callback(
    Output("correctActions", "children", allow_duplicate=True),
    Output("incorrectActions", "children", allow_duplicate=True),
    Output("config", "data", allow_duplicate=True),
    Output("indicators", "children", allow_duplicate=True),
    Output("cytoscape", "elements", allow_duplicate=True),
    Output("divSequence", "children", allow_duplicate=True),
    Output("correctness", "children", allow_duplicate=True),
    Input("makeStep", "n_clicks"),
    State("correctActions", "children"),
    State("incorrectActions", "children"),
    State("cytoscape","elements"),
    State("sequenceInput", "value"),
    State("config", "data"),
    State("indicators", "children"),
    State("divSequence", "children"),
    State("correctness", "children"),
    prevent_initial_call=True,
)
def makeStepButton(click,correctList,incorrectList, elements, sequence, config, indicators, divSequence, correctness):
    #print(divSequence)
    oldConfig = ""
    status = ""
    if (config == None):
        oldConfig = generateTerm(elements,encodeSequence(sequence.split(",")))
        config = makeStep(oldConfig)
    else:
        oldConfig = config
        config = makeStep(config)
    if (config != None):
        if (gentNewCorrectNumber(oldConfig, config)==None):
            divSequence = makeNewNumberRed(divSequence)
        if (gentNewIncorrectNumber(oldConfig, config)==None):
            divSequence = makeNewNumberGreen(divSequence)
        #print(gentNewCorrectNumber(oldConfig, config), " incorrect: " , gentNewIncorrectNumber(oldConfig, config))
        return getCorrectList(config).replace('a(-5)', 'a(S)').replace('a(-7)','a(F)'), getIncorrectList(config).replace('a(-5)', 'a(S)').replace('a(-7)','a(F)'), config, getTupleList(config), processTupleList(getTupleList(config), elements), divSequence, getStatus(config)
    else:
        return correctList, incorrectList, config, indicators, elements, divSequence, correctness
    
    
#button setFunction
@callback(
    #Output("cytoscape", "elements", allow_duplicate=True),
    Output("functionFlag", "data", allow_duplicate=True),
    Output("functionToggle", "children", allow_duplicate=True),
    Input("functionToggle", "n_clicks"),
    Input("functionFlag", "data"),
    prevent_initial_call = True, 
)
def buttonSetFunc(clicks, flag):
    if (flag == None) or (flag == False):
        flag = True
        return flag, "Stop Setting Node Functions"
    else:
        flag = False
        return flag, "Start Setting Node Functions"
    
#function button Pressed
@callback(
    Output("funcOR", "style", allow_duplicate=True),
    Output("funcXOR", "style", allow_duplicate=True),
    Output("funcAND", "style", allow_duplicate=True),
    Output("funcSEQ", "style", allow_duplicate=True),
    Output("nodeSelectedPreviously", "data", allow_duplicate=True),
    Output("cytoscape", "elements", allow_duplicate=True),
    State("nodeSelectedPreviously", "data"),
    Input("funcOR", "n_clicks"),
    Input("funcXOR", "n_clicks"),
    Input("funcAND", "n_clicks"),
    Input("funcSEQ", "n_clicks"),
    State("cytoscape", "elements"),
    prevent_initial_call = True,
)
def funButton(selectedNode, f_or, f_xor, f_and, f_seq, elements):
    if (selectedNode != None):
        button = ctx.triggered_id
        #print(button)
        match button:
            case "funcOR":
                elements = setNodeFunctionToOR(selectedNode, elements)
            case "funcXOR":
                elements = setNodeFunctionToXOR(selectedNode, elements)
            case "funcAND":
                elements = setNodeFunctionToAND(selectedNode, elements)
            case "funcSEQ":
                elements = setNodeFunctionToSEQ(selectedNode, elements)
        print(elements)
        return dict(display='none'), dict(display='none'), dict(display='none'), dict(display='none'), None, elements
    else:
        return buttonStyle, buttonStyle, buttonStyle, buttonStyle, None, elements
    

#button realtime
@callback(
    #Output("cytoscape", "elements", allow_duplicate=True),
    Output("realTimeFlag", "data", allow_duplicate=True),
    Output("realTime", "children", allow_duplicate=True),
    Input("realTime", "n_clicks"),
    State("realTimeFlag", "data"),
    prevent_initial_call = True, 
)
def buttonSetFunc(clicks, flag):
    #print('button pressed')
    if (flag == None) or (flag == False):
        flag = True
        return flag, "Stop Interactive Supervision"
    else:
        flag = False
        return flag, "Start Interactive Supervision"
    
#input
@callback(
    Output("divSequence", "children", allow_duplicate=True),
    Input("sequenceInput", "value"),
    prevent_initial_call = True,
)
def changeInputSequence(sequence):
    if (sequence == None):
        return sequence
    else:
        sequenceList = sequence.split(",")
        returnList = []
        for number in sequenceList:
            a = html.H4(children=str(number), style={'color': '#000000',  'margin-top':'5px', 'margin-bottom':'5px', 'margin-left':'0px', 'margin-right':'0px', 'padding':'0px', 'display': 'inline-block'}, title="unprocessed")
            b = html.H4(children=",", style={'color': '#000000',  'margin-top':'5px', 'margin-bottom':'5px', 'margin-left':'0px', 'margin-right':'0px', 'padding':'0px', 'display': 'inline-block'}, title="comma")
            #print(a.title)
            returnList.append(a)
            returnList.append(b)
        #print(returnList)
        return returnList[:-1]
        
app.run(debug=True)
#app.run(debug=True, host= "192.168.0.102", port=8053)


In [47]:
import maude


def initializeMaude():
    maude.init()
    maude.load('graphCookies.maude')
    module = maude.getCurrentModule()
    return module


def checkCorrectness(t):
    maude.init()
    maude.load('graphCookies.maude')
    module = maude.getCurrentModule()
    #module = initializeMaude()
    term = module.parseTerm(t)
    correctPattern = module.parseTerm('SL:ActionList | TL:TupleList | CL:ActionList | WL:ActionList | G:Graph | correct')
    for r, sb, ctx, rl in term.search(maude.ANY_STEPS, correctPattern):
        #print(r)
        return True
    return False


def checkIncorrectness(t):
    module = initializeMaude()
    term = module.parseTerm(t)
    incorrectPattern = module.parseTerm('SL:ActionList | TL:TupleList | CL:ActionList | WL:ActionList | G:Graph | incorrect')
    for r, sb, ctx, rl in term.search(maude.ANY_STEPS, incorrectPattern):
        return True
    return False

def makeStep(termString):
    #print(termString)
    module = initializeMaude()
    term = module.parseTerm(termString)
    pattern = module.parseTerm('SL:ActionList | TL:TupleList | CL:ActionList | WL:ActionList | G:Graph | S:Status')
    for r, sb, ctx, rl in term.search(0, pattern):
        return str(r)

def elementsToGraph(elements):
    graph = ""
    for element in elements:
        if 'source' in element['data'].keys():
            src = element['data']['source']
            dst = element['data']['target']
            graph = graph + '(n[' + src + ']: ' + getFunctionByID(src, elements) + ') -> (n[' + dst + ']: '+  getFunctionByID(dst, elements) +') ;; '
    return str(graph[:-4])

def getFunctionByID(ID, elements):
    for element in elements:
        if 'id' in element['data'].keys():
            if element['data']['id'] == ID:
                return element['data']['function']
           
def listToSequence(slist):
    seq = "("
    for element in slist:
        seq = seq + 'a(' + str(element) + ') ; '
    seq = seq[:-3] + ")"
    return str(seq)

def getInputList(term):
    return term.split('|')[0]
def getTupleList(term):
    return term.split('|')[1]
def getCorrectList(term):
    return term.split('|')[2]#.replace('a(-5)', 'a(S)').replace('a(-7)','a(F)')#.replace('a(', '').replace(')','').replace(';',',')
def getIncorrectList(term):
    return term.split('|')[3]#.replace('a(-5)', 'a(S)').replace('a(-7)','a(F)')
def getStatus(term):
    return term.split('|')[5]

def generateTerm(elements, actions):
    return "" + listToSequence(actions) + " | nil | nil | nil | " + elementsToGraph(elements) + " | starting"

def gentNewCorrectNumber(oldConfig,newConfig):
    if (getCorrectList(oldConfig) == getCorrectList(newConfig)):
        return None
    if (getCorrectList(oldConfig) == " nil "):
        return getCorrectList(newConfig).split("(")[1].split(")")[0]
    else:
        return getCorrectList(newConfig).replace(getCorrectList(oldConfig), "").split("(")[1].split(")")[0]

def gentNewIncorrectNumber(oldConfig,newConfig):
    if (getIncorrectList(oldConfig) == getIncorrectList(newConfig)):
        return None
    if (getIncorrectList(oldConfig) == " nil "):
        return getIncorrectList(newConfig).split("(")[1].split(")")[0]
    else:
        return getIncorrectList(newConfig).replace(getIncorrectList(oldConfig), "", 1).split("(")[1].split(")")[0]

def makeNewNumberRed(ilist):
    #print(ilist)
    for element in ilist:
        #print(element)
        if (element['props']['title'] == "unprocessed"):
            element['props']['style'] = {'color': '#FF0000', 'margin-top':'5px', 'margin-bottom':'5px', 'margin-left':'0px', 'margin-right':'0px', 'padding':'0px', 'display': 'inline-block'} 
            element['props']['title'] = "processed"
            print(ilist)
            return ilist
    return ilist
    #print ilist

def makeNewNumberGreen(ilist):
    #print(ilist)
    for element in ilist:
        #print(element)
        if (element['props']['title'] == "unprocessed"):
            element['props']['style'] = {'color': '#008000',  'margin-top':'5px', 'margin-bottom':'5px', 'margin-left':'0px', 'margin-right':'0px', 'padding':'0px', 'display': 'inline-block'} 
            element['props']['title'] = "processed"
            #print(ilist)
            return ilist
    return ilist
    #print ilist


[{'data': {'id': '5', 'label': '5', 'function': 'nseq', 'url': 'nseq.jpg'}, 'position': {'x': -123.98678613466984, 'y': -25.75959836336593}, 'classes': 'edge nseq'}, {'data': {'id': '1', 'label': '1', 'function': 'nseq'}, 'position': {'x': -199.69687989601323, 'y': -27.542949942975344}, 'classes': 'edge nseq'}, {'data': {'id': '2', 'label': '2', 'function': 'nor'}, 'position': {'x': 22.671122520389762, 'y': -23.69302613659202}, 'classes': 'edge nor'}, {'data': {'id': '3', 'label': '3', 'function': 'nxor'}, 'position': {'x': 122.9677742934759, 'y': -20.47176758837865}, 'classes': 'edge nxor'}, {'data': {'id': '-5', 'label': 'S', 'function': 'nseq'}, 'position': {'x': -268.32100254356936, 'y': -27.322884268868464}, 'classes': 'selected nseq'}, {'data': {'id': '8', 'label': '8', 'function': 'nseq'}, 'position': {'x': -50.57717883108286, 'y': -25.519417278496693}, 'classes': 'edge nseq'}, {'data': {'id': '6', 'label': '6', 'function': 'nor'}, 'position': {'x': 71.0986485417034, 'y': -73.12